<a href="https://colab.research.google.com/github/cccg8105/notebook_AD/blob/deep_learning/deep_learning/red_neuronal/RNA_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Red Neuronal Artificial (RNA)

## Instalar dependendias

In [1]:
!pip install sklearn
!pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git
!pip install keras
!pip install tensorflow

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-kuz_8k0r
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-kuz_8k0r
  Created wheel for Theano: filename=Theano-1.0.5+1.geb6a4125c-cp36-none-any.whl size=2668281 sha256=b228606517b019a59f869fb2c7aee39863437840a7267c5c4f4ab44deaf534d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-gu8l3dr2/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.5
    Uninstalling Theano-1.0.5:
      Successfully uninstalled Theano-1.0.5


## Importar conjunto de datos

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('https://raw.githubusercontent.com/cccg8105/deeplearning-az/master/datasets/Part%201%20-%20Artificial%20Neural%20Networks%20(ANN)/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

## Preprocesado de datos

### Conversión de datos categóricos

In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
)
X = onehotencoder.fit_transform(X)
X = X[:, 1:]

### Dividir en entrenamiento y prueba

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Escalado de variables

In [5]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Construcción

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta
nodosIniciales = 6 # se recomienda poner la media entre los nodos de entrada y salida (11+1)/2
inicializadorKernel = "uniform" # glorot_uniform|||
funcionActivacion = "relu" # funcion escalon|funcion sigmoide(sigmoid)|tangente hiperbolica(tanh)| rectificador lineal(relu)
nodosEntrada = 11 # numero de variables en la matriz de caracteristicas

classifier.add(Dense(units = nodosIniciales, kernel_initializer = inicializadorKernel,  
                     activation = funcionActivacion, input_dim = nodosEntrada))

# Añadir la segunda capa oculta
nodosCapa2 = 6
inicializadorKernel2 = "uniform" 
funcionActivacion2 = "relu" 
classifier.add(Dense(units = nodosCapa2, kernel_initializer = inicializadorKernel2,  activation = funcionActivacion2))

# Añadir la capa de salida
nodoSalida = 1
salidaKernel = "uniform" 
funcionActivacionSalida = "sigmoid" 
classifier.add(Dense(units = nodoSalida, kernel_initializer = salidaKernel,  activation = funcionActivacionSalida))

## Compilación

In [7]:
optimizador = "adam" # |||
funcionPerdida =  "binary_crossentropy" # softmax|||
metricas = ["accuracy"] # |||
classifier.compile(optimizer = optimizador, loss = funcionPerdida, metrics = metricas)

## Entrenamiento

In [8]:
tamanioLote = 10
epocas = 100
classifier.fit(X_train, y_train,  batch_size = tamanioLote, epochs = epocas)

Epoch 1/100
800/800 [==============================] - 1s 873us/step - loss: 0.4860 - accuracy: 0.7991
Epoch 2/100
800/800 [==============================] - 1s 883us/step - loss: 0.4093 - accuracy: 0.8236
Epoch 3/100
800/800 [==============================] - 1s 862us/step - loss: 0.3948 - accuracy: 0.8275
Epoch 4/100
800/800 [==============================] - 1s 881us/step - loss: 0.3842 - accuracy: 0.8285
Epoch 5/100
800/800 [==============================] - 1s 844us/step - loss: 0.3771 - accuracy: 0.8322
Epoch 6/100
800/800 [==============================] - 1s 857us/step - loss: 0.3714 - accuracy: 0.8436
Epoch 7/100
800/800 [==============================] - 1s 861us/step - loss: 0.3681 - accuracy: 0.8466
Epoch 8/100
800/800 [==============================] - 1s 873us/step - loss: 0.3652 - accuracy: 0.8493
Epoch 9/100
800/800 [==============================] - 1s 844us/step - loss: 0.3616 - accuracy: 0.8491
Epoch 10/100
800/800 [==============================] - 1s 875us/step - l

## Evaluar modelo con datos de prueba

In [11]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1539,   56],
       [ 222,  183]])

## Mejorar y ajustar el modelo

### Evaluación K-fold cross validation

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
  classifier = Sequential()
  classifier.add(Dense(units = nodosIniciales, kernel_initializer = inicializadorKernel, activation = funcionActivacion, input_dim = nodosEntrada))
  classifier.add(Dense(units = nodosCapa2, kernel_initializer = inicializadorKernel2, activation = funcionActivacion2))
  classifier.add(Dense(units = nodoSalida, kernel_initializer = salidaKernel, activation = funcionActivacionSalida))
  classifier.compile(optimizer = optimizador, loss = funcionPerdida, metrics = metricas)
  return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = tamanioLote, nb_epoch = epocas)

tamanioBloqueEntrenamiento = 10
tareasSimultaneas = -1 # usa todas las cpus de la pc
mostrarDatosIntermedios = 1
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = tamanioBloqueEntrenamiento, 
                             n_jobs= tareasSimultaneas, verbose = mostrarDatosIntermedios)

mean = accuracies.mean()
variance = accuracies.std()

print(mean)
print(variance)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.7985000073909759
0.009532839676923333


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    9.2s finished


- Alto sesgo, baja varianza:
- Alto sesgo, alta varianza:
- Bajo sesgo, baja varianza: tiene buena precision y poca varianza. Esto es lo mejor
- Bajo sesgo, alta varianza: buena precision pero mucha varianza. Es ideal disminuir la varianza

### Ajuste 

In [14]:
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
  classifier = Sequential()
  classifier.add(Dense(units = nodosIniciales, kernel_initializer = inicializadorKernel,  activation = funcionActivacion, input_dim = nodosEntrada))
  classifier.add(Dense(units = nodosCapa2, kernel_initializer = inicializadorKernel2,  activation = funcionActivacion2))
  classifier.add(Dense(units = nodoSalida, kernel_initializer = salidaKernel,  activation = funcionActivacionSalida))
  classifier.compile(optimizer = optimizer, loss = funcionPerdida, metrics = metricas)
  return classifier

classifier = KerasClassifier(build_fn = build_classifier)

rangoLotes = [25,32]
rangoEpocas = [100, 500]
optimizadores = ['adam', 'rmsprop']
parameters = {
    'batch_size' : rangoLotes,
    'nb_epoch' : rangoEpocas, 
    'optimizer' : optimizadores
}

grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

288/288 [==============================] - 0s 963us/step - loss: 0.5494 - accuracy: 0.7968
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
320/320 [==============================] - 0s 947us/step - loss: 0.5325 - accuracy: 0.7960


In [15]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'nb_epoch': 100, 'optimizer': 'adam'}
0.796
